# Risk Aggregation POC - ELTs

This notebook reads can be used to read in multiple ELTs, rollup the ELTs to an aggregate ELTs, and generate statistics and EP metrics from the ELTs.

-----

### Install necessary libraries

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install pyarrow
# %pip install fastparquet
# %pip install rpy2
# %pip install scipy==1.6.0
# Install to connect to AWS S3
# %pip install boto3
# %pip install botocore==1.22.5
# %pip install s3fs
# %pip install fsspec

In [2]:
### Import necessary libraries

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import aggregationtools.elt
import aggregationtools
from aggregationtools.elt import ELT
from aggregationtools import elt_calculator
import glob

### Import data from AWS S3
AWS credentials may be provided explicitly with s3fs.S3FileSystem,
but it is more secure to exclude the credentials from the code.
Instead use the AWS CLI to `aws configure` credentials.

In [4]:
# import glob
# import s3fs

# s3 = s3fs.S3FileSystem(anon=False)
# files = s3.glob('s3://fannie-mae-phase-3/03 Data Aggregation POC/01 Test/SCENARIO_DATA/EARTHQUAKE_CA_NY_FL/*.parquet')
# elts = pd.concat([pd.read_parquet('s3://' + fp) for fp in files])

### Import local data

In [5]:
path = os.path.join('AggregationPOCScenarios', 'EARTHQUAKE_CA_NY_FL', '')
files = glob.glob(path + '*.parquet')
elts = pd.concat([pd.read_parquet(fp) for fp in files])

### Create ELT from imported data

In [6]:
# json_test = plts.to_dict(orient='records')
original_elt = aggregationtools.elt.ELT(data = elts)
original_elt.elt.head(5).style

,AccgrpId,LocName,Lob,EventId,Rate,Loss,StdDevI,StdDevC,ExpValue,StandardDev,mu,sigma,alpha,beta
0,3923,0002c2a852d9c0a281fed88d8886d92459e,ATC43,15437392,0.000000,1254.516148,2619.420310,555.634611,201601.626126,3175.054921,0.006223,0.015749,0.148923,23.783062
1,3923,0002c2a852d9c0a281fed88d8886d92459e,ATC43,15437387,0.000000,129.254763,277.524609,58.868857,201448.770241,336.393466,0.000642,0.001670,0.146902,228.805171
2,3923,0002c2a852d9c0a281fed88d8886d92459e,ATC43,15434262,0.000000,656.991289,1392.321647,295.340955,200730.347584,1687.662603,0.003273,0.008408,0.147778,45.002850
3,3923,0002c2a852d9c0a281fed88d8886d92459e,ATC43,15434259,0.000000,3199.460520,6347.416511,1346.421684,199354.644623,7693.838195,0.016049,0.038594,0.154104,9.447960
4,3923,0002c2a852d9c0a281fed88d8886d92459e,ATC43,15434257,0.000000,714.610089,1512.091439,320.746669,199507.500508,1832.838108,0.003582,0.009187,0.147890,41.140561


### Group ELTs

In [7]:
grouped_elt = elt_calculator.group_elts(original_elt)
grouped_elt.elt.head(5).style

,EventId,Rate,Loss,StdDevI,StdDevC,ExpValue,StandardDev,mu,sigma,alpha,beta
0,15072596,0.000020,1854.150706,3535.919802,883.979950,99999.990000,4419.899752,0.018542,0.044199,0.154176,8.161022
1,15072380,0.000034,1080.485117,2146.784497,536.696124,99999.990000,2683.480621,0.010805,0.026835,0.149565,13.692824
2,15072306,0.000050,69.499806,145.338224,36.334556,99999.990000,181.672780,0.000695,0.001817,0.145552,209.281652
3,15072904,0.000015,6.536827,13.712308,3.428077,99999.990000,17.140385,0.000065,0.000171,0.145368,2223.687603
4,15072502,0.000029,232.392318,482.073016,120.518254,99999.990000,602.591270,0.002324,0.006026,0.146060,62.704632


### Calculate statistics from grouped ELT

In [8]:
aal = grouped_elt.get_aal()
std = grouped_elt.get_standard_deviation()
grouped_elt.elt.style
covvar = grouped_elt.get_covvar()
print('AAL: ' + str(aal))
print('STD: ' + str(std))
print('CovVar: ' + str(covvar))

AAL: 220977.9676848252
STD: 871605.8685159368
CovVar: 3.9443111801945983


### Calculate AEP & OEP metrics from ELT

In [9]:
oep = elt_calculator.calculate_oep_curve(grouped_elt.elt)
standard_rp_oep = oep.get_standard_return_period_ep()
standard_rp_oep = pd.DataFrame.from_dict([standard_rp_oep]).T
standard_rp_oep = standard_rp_oep[standard_rp_oep.index.isin([1 / x for x in oep.RETURN_PERIODS])]
standard_rp_oep = standard_rp_oep.rename_axis("Return Period").sort_index(axis=0, ascending=True)
standard_rp_oep.columns = ["Loss"]
standard_rp_oep.index = np.reciprocal(standard_rp_oep.index).to_series().apply(lambda x: np.round(x,2))
# Return OEP results
standard_rp_oep.style.format("{:,.0f}")

,Loss
Return Period,
50000.0,"28,580,351"
10000.0,"18,417,082"
5000.0,"15,615,291"
1000.0,"10,202,752"
500.0,"8,075,735"
250.0,"6,133,646"
200.0,"5,562,561"
100.0,"3,943,111"
50.0,"2,504,840"


In [10]:
aep = elt_calculator.calculate_aep_curve(grouped_elt.elt)
standard_rp_aep = aep.get_standard_return_period_ep()
standard_rp_aep = pd.DataFrame.from_dict([standard_rp_aep]).T
standard_rp_aep = standard_rp_aep[standard_rp_aep.index.isin([1 / x for x in aep.RETURN_PERIODS])]
standard_rp_aep = standard_rp_aep.rename_axis("Return Period").sort_index(axis=0, ascending=True)
standard_rp_aep.columns = ["Loss"]
standard_rp_aep.index = np.reciprocal(standard_rp_aep.index).to_series().apply(lambda x: np.round(x,2))
# Return AEP results
standard_rp_aep.style.format("{:,.0f}")

,Loss
Return Period,
50000.0,"28,934,697"
10000.0,"18,810,567"
5000.0,"15,983,321"
1000.0,"10,505,096"
500.0,"8,352,352"
250.0,"6,372,944"
200.0,"5,787,078"
100.0,"4,120,424"
50.0,"2,639,826"
